## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
import os, sys
sys.path.insert(0, '..')

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,60.01,78,96,1.99,overcast clouds
1,1,Avera,US,33.1940,-82.5271,68.00,32,40,5.75,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,66.45,72,0,21.50,clear sky
3,3,San Cristobal,VE,7.7669,-72.2250,84.20,65,20,8.05,few clouds
4,4,Pisco,PE,-13.7000,-76.2167,66.99,82,0,11.50,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 100


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avera,US,33.1940,-82.5271,68.00,32,40,5.75,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,66.45,72,0,21.50,clear sky
3,3,San Cristobal,VE,7.7669,-72.2250,84.20,65,20,8.05,few clouds
4,4,Pisco,PE,-13.7000,-76.2167,66.99,82,0,11.50,clear sky
11,11,Saint-Philippe,RE,-21.3585,55.7679,77.00,78,20,13.80,light rain
13,13,Lindi,TZ,-10.0000,39.7167,72.32,96,62,8.05,broken clouds
18,18,Hilo,US,19.7297,-155.0900,82.40,54,40,12.66,light rain
19,19,Oriximina,BR,-1.7656,-55.8661,76.10,92,86,3.22,overcast clouds
20,20,Rikitea,PF,-23.1203,-134.9692,78.75,68,100,17.11,overcast clouds
21,21,Cap Malheureux,MU,-19.9842,57.6142,77.00,82,99,1.99,light rain


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                325
City                   325
Country                324
Lat                    325
Lng                    325
Max Temp               325
Humidity               325
Cloudiness             325
Wind Speed             325
Current Description    325
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avera,US,33.1940,-82.5271,68.00,32,40,5.75,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,66.45,72,0,21.50,clear sky
3,3,San Cristobal,VE,7.7669,-72.2250,84.20,65,20,8.05,few clouds
4,4,Pisco,PE,-13.7000,-76.2167,66.99,82,0,11.50,clear sky
11,11,Saint-Philippe,RE,-21.3585,55.7679,77.00,78,20,13.80,light rain


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avera,US,68.00,scattered clouds,33.1940,-82.5271,
2,Port Alfred,ZA,66.45,clear sky,-33.5906,26.8910,
3,San Cristobal,VE,84.20,few clouds,7.7669,-72.2250,
4,Pisco,PE,66.99,clear sky,-13.7000,-76.2167,
11,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
13,Lindi,TZ,72.32,broken clouds,-10.0000,39.7167,
18,Hilo,US,82.40,light rain,19.7297,-155.0900,
19,Oriximina,BR,76.10,overcast clouds,-1.7656,-55.8661,
20,Rikitea,PF,78.75,overcast clouds,-23.1203,-134.9692,
21,Cap Malheureux,MU,77.00,light rain,-19.9842,57.6142,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
#Display hotel_df
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avera,US,68.00,scattered clouds,33.1940,-82.5271,
2,Port Alfred,ZA,66.45,clear sky,-33.5906,26.8910,The Halyards Hotel
3,San Cristobal,VE,84.20,few clouds,7.7669,-72.2250,Posada Villaven C.A.
4,Pisco,PE,66.99,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
11,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
694,Miraflores,MX,80.60,scattered clouds,19.2167,-98.8083,Hotel Mansión
697,Conceicao Da Barra,BR,73.38,clear sky,-18.5933,-39.7322,Cricaré Praia Hotel
701,Tezu,IN,70.02,broken clouds,27.9167,96.1667,Circuit House
702,Mahibadhoo,MV,83.26,scattered clouds,3.7833,72.9667,Omadhoo Inn


In [19]:
# 7. Drop the rows where there is no Hotel Name.
#import numpy 
import numpy as np
#convert empty string in Hotel Name to np.nan
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
#Drop rows with no Hotel Name
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,66.45,clear sky,-33.5906,26.8910,The Halyards Hotel
3,San Cristobal,VE,84.20,few clouds,7.7669,-72.2250,Posada Villaven C.A.
4,Pisco,PE,66.99,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
11,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Lindi,TZ,72.32,broken clouds,-10.0000,39.7167,Lindi paradise house
...,...,...,...,...,...,...,...
694,Miraflores,MX,80.60,scattered clouds,19.2167,-98.8083,Hotel Mansión
697,Conceicao Da Barra,BR,73.38,clear sky,-18.5933,-39.7322,Cricaré Praia Hotel
701,Tezu,IN,70.02,broken clouds,27.9167,96.1667,Circuit House
702,Mahibadhoo,MV,83.26,scattered clouds,3.7833,72.9667,Omadhoo Inn


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

#11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))